<a href="https://colab.research.google.com/github/semishen/DL-CVMarathon/blob/master/Day015_Cifar_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Activation

import keras.utils
from keras.datasets import cifar10

import numpy as np
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [4]:
print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0,1,2,3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test, mean, std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [0]:
# ## OneHot Label 由(None, 1)-(None, 10)
# ## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
# one_hot=OneHotEncoder()
# y_train=one_hot.fit_transform(y_train).toarray()
# y_test=one_hot.transform(y_test).toarray()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [6]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(kernel_size=(3,3), filters=16, input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(kernel_size=(3,3), filters=32))
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100)) #output_dim=100,activation=relu
classifier.add(Activation('relu'))


#輸出
classifier.add(Dense(units=10,activation='softmax'))
classifier.summary()

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 16)        448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 13, 13, 32)       

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [25]:
index = 50
input_example = x_test[index].reshape(1,32,32,3)
# input_example.shape
input_example=(input_example-mean_train)/(std_train+1e-7) 
print(classifier.predict(input_example))
print(y_test[index])

[[9.8900187e-01 4.3648228e-18 1.9784315e-05 1.0233850e-02 6.6645944e-04
  9.5122732e-10 1.2182406e-09 1.8907217e-06 7.6206437e-05 2.0525989e-18]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
